In [13]:
import apriltag
import cv2
import numpy as np
import math
import mediapipe as mp

In [14]:
BoxDims = []
def CreateUI():
	UIHeight = 1500
	UIWidth = 1000

	UIBG = np.zeros((UIHeight,UIWidth,3),dtype=np.uint8)

	BoxHeight = 1000
	BoxWidth = 200
	Gap = 100

	YStart = int((UIHeight-BoxHeight)/2)

	for i in range(3):
		XStart = Gap + i*(BoxWidth+Gap)

		BoxTopLeft = (XStart,YStart)
		BoxBottomRight = (XStart+BoxWidth , YStart+BoxHeight)

		cv2.rectangle(UIBG,BoxTopLeft,BoxBottomRight,(192,242,30),-1)

		BoxDims.append((XStart,YStart,BoxWidth,BoxHeight))
	return UIBG

In [15]:
#Setup

UIBGOriginal = CreateUI()
h,w,c = UIBGOriginal.shape
options = apriltag.DetectorOptions(families="tag16h5")
detector = apriltag.Detector(options)
Angle = 0
mphands = mp.solutions.hands
mpdrawing = mp.solutions.drawing_utils
hand = mphands.Hands(min_detection_confidence = 0.7,max_num_hands = 1)

cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
if not cam.isOpened():
	print("Cannot Open Camera")
	exit()

I0000 00:00:1757855926.132628  413232 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757855926.195859  498524 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 580.65.06), renderer: NVIDIA GeForce RTX 5070 Laptop GPU/PCIe/SSE2
W0000 00:00:1757855926.211023  498501 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757855926.226188  498522 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [16]:
while True:
	success,frame = cam.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	HFrame,WFrame,_ = frame.shape

	ApriltagResults = detector.detect(gray)

	Matrix = None
	FingerOption = -1
	UIBG = UIBGOriginal.copy()
	
	if ApriltagResults:
		
		Tag = ApriltagResults[0]
		MarkerCorners = Tag.corners

		MarkerTopLeft,MarkerTopRight,MarkerBottomRight,MarkerBottomLeft = MarkerCorners

		TopVector = MarkerTopRight - MarkerTopLeft
		LeftVector = MarkerBottomLeft - MarkerTopLeft

		OffsetScale = 0
		UIScale = 5

		UITopLeft = MarkerBottomLeft+(LeftVector*OffsetScale)
		UITopRight = UITopLeft+(TopVector*UIScale)
		UIBottomLeft = UITopLeft+(LeftVector*UIScale)
		UIBottomRight = UITopLeft+(TopVector*UIScale)+(LeftVector*UIScale)

		DestinationPoints = np.array([UITopLeft,UITopRight,UIBottomRight,UIBottomLeft], dtype=np.float32)
		SourcePoints = np.array([[0,0],[w,0],[w,h],[0,h]], dtype=np.float32)
		Matrix,_ = cv2.findHomography(SourcePoints,DestinationPoints)

		cv2.polylines(gray, [np.int32(DestinationPoints)], True, 255, 3)

	RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	MpResult = hand.process(RGB)
	Pinching = False

	if MpResult.multi_hand_landmarks and Matrix is not None:
		HandLandmarks = MpResult.multi_hand_landmarks[0]
		mpdrawing.draw_landmarks(frame, HandLandmarks, mphands.HAND_CONNECTIONS)

		IndexTip = HandLandmarks.landmark[mphands.HandLandmark.INDEX_FINGER_TIP]
		ThumbTip = HandLandmarks.landmark[mphands.HandLandmark.THUMB_TIP]

		IndexPos = (int(IndexTip.x * WFrame),int(IndexTip.y * HFrame))

		try:
			InvMatrix = np.linalg.inv(Matrix)
			FingerPosOnUI = cv2.perspectiveTransform(np.array([[IndexPos]], dtype=np.float32), InvMatrix)
			fx,fy = int(FingerPosOnUI[0][0][0]),int(FingerPosOnUI[0][0][1])

			cv2.circle(UIBG, (fx, fy), 15, (0, 0, 255), -1)

			for i,(bx,by,bw,bh) in enumerate(BoxDims):
				print(f"Box Dimensions: {(bx,by,bw,bh)}")
				if bx<fx<bx+bw and by<fy<by+bh:
					FingerOption = i
					print(f"FingerOption has been updated to: {FingerOption}")
					break
		except np.linalg.LinAlgError:
			print("LinAlgError!")

		dist = math.hypot(ThumbTip.x - IndexTip.x , ThumbTip.y-IndexTip.y)
		print(f"Distance between fingers: {dist}")
		IsPinching = dist < 0.1

		if FingerOption != -1 and IsPinching:
			print(f"User selected box {FingerOption+1}")
			(bx,by,bw,bh) = BoxDims[FingerOption]
			cv2.rectangle(UIBG, (bx,by), (bx+bw,by+bh), (0,255,0), -1)
			Pinching = True

	if not Pinching:
		Pinching = False

	if Matrix is not None:
		WarpedUI = cv2.warpPerspective(UIBG, Matrix, (WFrame,HFrame))

		mask = np.sum(WarpedUI, axis=2) > 0
		frame[mask] = WarpedUI[mask]

	cv2.imshow("VideoFeed",frame)
	cv2.imshow("MediaPipeBTS",RGB)
	cv2.imshow("AprilTagBTS",gray)

	if cv2.waitKey(1) == ord('q'):
		break

cam.release()
cv2.destroyAllWindows()

/tmp/pip-install-fhwy8tae/apriltag_637ea0c35a5445cb8841ea69e77a8f97/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.10554871954888018
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.14540357593202421
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.12703640477357778
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.1087475400858273
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.10834893916894493
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.10373301454922221
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.014596252750051988
User selected box 1
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.020464472671030572
User selected box 1

/tmp/pip-install-fhwy8tae/apriltag_637ea0c35a5445cb8841ea69e77a8f97/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
Distance between fingers: 0.049213543735216796
Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
Distance between fingers: 0.050367839881256427
Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
FingerOption has been updated to: 2
Distance between fingers: 0.050174291696041984
User selected box 3
Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
FingerOption has been updated to: 2
Distance between fingers: 0.04825616397373312
User selected box 3
Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
FingerOption has been updated to: 2
Distance between fingers: 0.05219429778066442
User selected box 3
Box Dimensions: (100, 250, 20

/tmp/pip-install-fhwy8tae/apriltag_637ea0c35a5445cb8841ea69e77a8f97/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
FingerOption has been updated to: 1
Distance between fingers: 0.03572705254291937
User selected box 2
Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
Distance between fingers: 0.018108005026380703
Box Dimensions: (100, 250, 200, 1000)
FingerOption has been updated to: 0
Distance between fingers: 0.047070169268954694
User selected box 1
Box Dimensions: (100, 250, 200, 1000)
Box Dimensions: (400, 250, 200, 1000)
Box Dimensions: (700, 250, 200, 1000)
Distance between fingers: 0.03201225560513002
